In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
from image_transformations import img_trans
import random
import tensorflow as tf

mnist = tf.keras.datasets.mnist

c:\users\tdhar\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
(X_train, y_train),(X_test, y_test) = mnist.load_data()

In [3]:
def up_dwn_brightness(data1, data2, data3, scale):
    new_x_train  = []
    new_x_test   = []
    new_y_train  = []
    new_y_test   = []
    
    for image in data1:
        up_dwn_data.append(image)
        
    for image in data2:
        value_change = random.randint(0,int(round(255*scale)))   

        up_lim = 255 - value_change
        image[0][image[0] > up_lim] = 255
        image[0][image[0] <= up_lim] += value_change
        
        up_dwn_data.append(image)
        
    for image in data3:
        value_change = random.randint(0,int(round(255*scale)))   

        image[0][image[0] < value_change] = 0
        image[0][image[0] >= value_change] -= value_change
        
        up_dwn_data.append(image)

    return up_dwn_data

In [4]:
"""
Things we care about:
brightness, rotation, blur

What this results is a dataset that is three times the size: it is the original dataset with an extra image up and down.
"""
def up_dwn(x, y, parameter, scale):
    new_x  = []
    new_y  = []

    for img, label in zip(x, y):
        
        if parameter == 'rotation':
            new_x.append(img)
            
            value_change = random.randint(0,int(round(180*scale)))

            up_img = img_trans(img).change_angle(value_change)
            dwn_img = img_trans(img).change_angle(-value_change)
            
            new_x.append(up_img)
            new_x.append(dwn_img)  
            
            new_y += 3 * [label]

        if parameter == 'blur':
            new_x.append(img)

            value_change  = random.randrange(3, 9, 2)
            sigma1        = random.randrange(3, 9, 2)
            value_change2 = random.randrange(3, 9, 2)
            sigma2        = random.randrange(3, 9, 2)
            
            
            img1 = img_trans(img).gaussian_blur(value_change,sigma1)
            img2 = img_trans(img).gaussian_blur(value_change2,sigma2)
            
            new_x.append(img1)
            new_x.append(img2)  
            
            new_y += 3 * [label]           
    
    
    return np.array(new_x), np.array(new_y)

In [7]:
parameter_list = ['rotation', 'blur']
scale_list = [.33,.67,1]

for parameter_in in parameter_list:
    for scale_in in scale_list:
        NAME = 'mnist_{}_{}.pickle'.format(parameter_in, str(scale_in))
        
        new_X_train, new_y_train = up_dwn(X_train, y_train,
                                          parameter = parameter_in,
                                          scale = scale_in)
        
        new_X_test, new_y_test = up_dwn(X_test, y_test,
                                        parameter = parameter_in,
                                        scale = scale_in) 
        
        total = [new_X_train, new_y_train, new_X_test, new_y_test]
        
        outfile = open(NAME, 'wb')
        pickle.dump(total, outfile)
        outfile.close()